# Libraries

In [1]:
#!conda install -c pytorch faiss-cpu -y

import pandas as pd
import faiss
import os
import numpy as np
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM, pipeline, AutoModelForSequenceClassification, T5EncoderModel


Retrieving notices: ...working... done
Channels:
 - pytorch
 - rapidsai
 - nvidia
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: \ | / - \ | / - \ | / - \ done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - faiss-cpu


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-24.7.1               |  py310hff52083_0         940 KB  conda-forge
    faiss-cpu-1.8.0            |py3.10_h9d89a2e_0_cpu         4.4 MB  pytorch
    libfaiss-1.8.0             |   hf65b397_0_cpu         5.2 MB  pytorch
    mkl-2023.2.0               |   h84fe81f_50496       156.8 MB  conda-forge
    openssl-3.3.1              |       h4bc722e_2         2.8 MB  conda-forge
    ------------------------------------------------------------
                                           Total:       170.0 MB

The following NEW packages will b

2024-08-17 05:35:24.444214: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-17 05:35:24.444348: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-17 05:35:24.598588: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Preprocessing of web logs

In [2]:
file_path = 'https://drive.google.com/uc?id=1ktNqoqwSOfNUg4MTQCJ27mGYJwgFilmB'
df = pd.read_csv(file_path, delim_whitespace=True, header=None, 
                 names=["IP Address", "-", "--", "Timestamp", "HTTP Method", "URL", "HTTP Version", 
                        "Status Code", "Response Size", "Referer", "User Agent"])
df['Timestamp'] = df['Timestamp'].str.strip('[]')

/tmp/ipykernel_18/1904017543.py:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(file_path, delim_whitespace=True, header=None,


In [3]:
df.head()

,IP Address,-,--,Timestamp,HTTP Method,URL,HTTP Version,Status Code,Response Size,Referer,User Agent
0,192.168.1.68,-,-,10/Aug/2024:06:12:33,+0000],POST /index.html HTTP/1.1,404,1696,http://example.com/products,Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like M...,NaN
1,192.168.1.17,-,-,07/Aug/2024:07:28:33,+0000],POST /index.html HTTP/1.1,302,1785,http://example.com/products,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,NaN
2,192.168.1.64,-,-,06/Aug/2024:11:22:33,+0000],POST /user/profile HTTP/1.1,200,339,http://example.com/index.html,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,NaN
3,192.168.1.44,-,-,10/Aug/2024:00:04:33,+0000],POST /user/profile HTTP/1.1,200,1838,-,Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90....,NaN
4,192.168.1.76,-,-,08/Aug/2024:15:20:33,+0000],GET /index.html HTTP/1.1,404,509,http://example.com/index.html,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,NaN


In [4]:
temp_df = df.loc[:, ['HTTP Version', 'Status Code', 'Response Size', 'Referer', 'User Agent']]
temp_df.head()

,HTTP Version,Status Code,Response Size,Referer,User Agent
0,404,1696,http://example.com/products,Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like M...,NaN
1,302,1785,http://example.com/products,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,NaN
2,200,339,http://example.com/index.html,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,NaN
3,200,1838,-,Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90....,NaN
4,404,509,http://example.com/index.html,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,NaN


In [5]:
df[['HTTP Method', 'URL', 'HTTP Version']] = df['URL'].str.split(' ', expand=True)

In [6]:
df['Status Code'] = temp_df['HTTP Version']
df['Response Size'] = temp_df['Status Code']
df['Referer'] = temp_df['Response Size']
df['User Agent'] = temp_df['Referer']
df.drop(['IP Address', '-', '--','Timestamp', 'Referer'], axis=1, inplace=True)

In [7]:
df.head()

,HTTP Method,URL,HTTP Version,Status Code,Response Size,User Agent
0,POST,/index.html,HTTP/1.1,404,1696,Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like M...
1,POST,/index.html,HTTP/1.1,302,1785,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...
2,POST,/user/profile,HTTP/1.1,200,339,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...
3,POST,/user/profile,HTTP/1.1,200,1838,Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90....
4,GET,/index.html,HTTP/1.1,404,509,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...


# RAG models

In [8]:
class LogProcessor:
    def __init__(self, df, model_name, tokenizer, model,chunk_size=512, overlap_size=50):
        self.df = df
        self.model_name = model_name
        self.chunk_size = chunk_size
        self.overlap_size = overlap_size

        self.tokenizer = tokenizer
        self.model = model
        
        self.base_dir = f"outputs_gptNemo/{model_name}"
        self.log_vectors_file = os.path.join(self.base_dir, f"{model_name}_log_vectors.npy")
        self.chunked_logs_file = os.path.join(self.base_dir, f"{model_name}_chunked_logs.pkl")

        os.makedirs(self.base_dir, exist_ok=True)

    def chunk_text(self, text):
        chunks = []
        start = 0
        while start < len(text):
            end = min(start + self.chunk_size, len(text))
            chunks.append(text[start:end])
            start += self.chunk_size - self.overlap_size
        return chunks

    def log_to_vector_no_logits(self, log_entry):
        if self.tokenizer.pad_token is None:
            self.tokenizer.add_special_tokens({'pad_token': '[PAD]'})
        inputs = self.tokenizer(log_entry, return_tensors='pt', padding=True, truncation=True)
        outputs = self.model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1).detach().numpy()
        return embeddings
    
    def log_to_vector_logits(self, log_entry):
        if self.tokenizer.pad_token is None:
            self.tokenizer.add_special_tokens({'pad_token': '[PAD]'})
        inputs = self.tokenizer(log_entry, return_tensors='pt', padding=True, truncation=True)
        outputs = self.model(**inputs)
        embeddings = outputs.logits.mean(dim=1).detach().numpy()
        return embeddings

    def preprocess_logs(self):
        logs = self.df.apply(lambda row: f"{row['HTTP Method']} {row['URL']} {row['HTTP Version']} {row['Status Code']} {row['Response Size']} {row['User Agent']}", axis=1)
        all_chunks = []
        for log in logs:
            chunks = self.chunk_text(log)
            all_chunks.extend(chunks)
        return all_chunks

    def create_faiss_index(self, vectors):
        dimension = vectors.shape[1]
        index = faiss.IndexFlatL2(dimension)
        index.add(vectors)
        return index

    def retrieve_logs_no_logits(self, query_chunks, index, logs, top_k=5):
        all_retrieved_logs = []
        for chunk in query_chunks:
            query_vector = self.log_to_vector_no_logits(chunk)
            _, indices = index.search(query_vector, top_k)
            retrieved_logs = [logs[i] for i in indices[0]]
            all_retrieved_logs.extend(retrieved_logs)
        return list(set(all_retrieved_logs))
    
    def retrieve_logs_logits(self, query_chunks, index, logs, top_k=5):
        all_retrieved_logs = []
        for chunk in query_chunks:
            query_vector = self.log_to_vector_logits(chunk)
            distances, indices = index.search(query_vector, top_k)
            retrieved_logs = [logs[i] for i in indices[0]]
            all_retrieved_logs.extend(retrieved_logs)
        return list(set(all_retrieved_logs))

    def process_no_logits(self):
        if os.path.exists(self.chunked_logs_file) and os.path.exists(self.log_vectors_file):
            chunked_logs = pd.read_pickle(self.chunked_logs_file)
            log_vectors = np.load(self.log_vectors_file)
        else:
            chunked_logs = self.preprocess_logs()
            log_vectors = np.vstack([self.log_to_vector_no_logits(chunk) for chunk in chunked_logs])
            pd.to_pickle(chunked_logs, self.chunked_logs_file)
            np.save(self.log_vectors_file, log_vectors)
        
        index = self.create_faiss_index(log_vectors)
        return index, chunked_logs

    def process_logits(self):
        if os.path.exists(self.chunked_logs_file) and os.path.exists(self.log_vectors_file):
            chunked_logs = pd.read_pickle(self.chunked_logs_file)
            log_vectors = np.load(self.log_vectors_file)
        else:
            chunked_logs = self.preprocess_logs()
            log_vectors = np.vstack([self.log_to_vector_logits(chunk) for chunk in chunked_logs])
            pd.to_pickle(chunked_logs, self.chunked_logs_file)
            np.save(self.log_vectors_file, log_vectors)
        
        index = self.create_faiss_index(log_vectors)
        return index, chunked_logs
    
    def generate_answer(self, retrieved_logs, query, generator):
        context = "\n".join(retrieved_logs)
        prompt = f"Context: {context}\n\nQuestion: {query}\nAnswer:"
        response = generator(prompt, max_new_tokens=50, num_return_sequences=1)
        return response[0]['generated_text']


In [9]:
queries = ["How many requests resulted in a 404 status code?", 
           "What is the most common HTTP method?", 
           "What is the most common status code?", 
           "What is the most common user agent?",
           "Which User Agent was used most frequently?",
           "What is the average response size for requests to /user/profile?",
           "How many unique URLs were accessed?",
           "Which HTTP method was used most often for successful requests (status code 200)?"]

## Vectorize: sentence-transformers/all-MiniLM-L6-v2  
## Generator: EleutherAI/gpt-neo-1.3B

In [10]:
tokenizer_l6 = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model_l6 = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

gptNeo_tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")
gptNeo_model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B")
generator_gptNeo = pipeline('text-generation', model=gptNeo_model, tokenizer=gptNeo_tokenizer)

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.31G [00:00<?, ?B/s]

In [11]:
processor_l6_gptNeo= LogProcessor(df,model_name="all-MiniLM-L6-v2", tokenizer=tokenizer_l6, model=model_l6)

index_l6_gptNeo, chunked_logs_l6_gptNeo = processor_l6_gptNeo.process_no_logits()
query = queries[-1]
chunked_query_l6_gptNeo = processor_l6_gptNeo.chunk_text(query)
retrieved_logs_l6_gptNeo = processor_l6_gptNeo.retrieve_logs_no_logits(chunked_query_l6_gptNeo, index_l6_gptNeo, chunked_logs_l6_gptNeo)

In [12]:
processor_l6_gptNeo.generate_answer(retrieved_logs_l6_gptNeo, query, generator_gptNeo)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


"Context: GET /about HTTP/1.1 302 1016 Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0\nGET /about HTTP/1.1 302 1135 Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0\nGET /about HTTP/1.1 500 1998 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36\nGET /about HTTP/1.1 200 4019 Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0\nGET /about HTTP/1.1 500 1834 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36\n\nQuestion: Which HTTP method was used most often for successful requests (status code 200)?\nAnswer: Request method was GET.\n\nA:\n\nThe GET method was most often used, by default, when you don't need to make the response bigger.\nHTTP/1.1 (301)\nMozilla/5.0"

## Vectorize: bert-base-uncased
## Generator: EleutherAI/gpt-neo-1.3B

In [13]:
bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
bert_model = AutoModel.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [14]:
processor_bert_gptNeo= LogProcessor(df,model_name="bert", tokenizer=bert_tokenizer, model=bert_model)

index_bert_gptNeo, chunked_logs_bert_gptNeo = processor_bert_gptNeo.process_no_logits()
query = queries[-1]
chunked_query_bert_gptNeo = processor_bert_gptNeo.chunk_text(query)
retrieved_logs_bert_gptNeo = processor_bert_gptNeo.retrieve_logs_no_logits(chunked_query_bert_gptNeo, index_bert_gptNeo, chunked_logs_bert_gptNeo)

In [15]:

processor_bert_gptNeo.generate_answer(retrieved_logs_bert_gptNeo, query, generator_gptNeo)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Context: DELETE /about HTTP/1.1 302 2882 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1\nDELETE /about HTTP/1.1 302 3203 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1\nDELETE /checkout HTTP/1.1 302 2547 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1\nDELETE /settings HTTP/1.1 404 2674 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1\nDELETE /about HTTP/1.1 302 754 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1\n\nQuestion: Which HTTP method was used most often for successful requests (status code 200)?\nAnswer:\n\nPostman\n\nRequest Headers:\nConten

## Vectorize: distilbert-base-uncased
## Generator: EleutherAI/gpt-neo-1.3B

In [16]:
distilbert_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
distilbert_model = AutoModel.from_pretrained("distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [17]:
processor_distilbert_gptNeo= LogProcessor(df,model_name="distilbert", tokenizer=distilbert_tokenizer, model=distilbert_model)

index_distilbert_gptNeo, chunked_logs_distilbert_gptNeo = processor_distilbert_gptNeo.process_no_logits()
query = queries[-1]
chunked_query_distilbert_gptNeo = processor_distilbert_gptNeo.chunk_text(query)
retrieved_logs_distilbert_gptNeo = processor_distilbert_gptNeo.retrieve_logs_no_logits(chunked_query_distilbert_gptNeo, index_distilbert_gptNeo, chunked_logs_distilbert_gptNeo)

In [18]:
processor_distilbert_gptNeo.generate_answer(retrieved_logs_distilbert_gptNeo, query, generator_gptNeo)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


"Context: GET /products?id=6417 HTTP/1.1 302 1928 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1\nPUT /checkout HTTP/1.1 404 1985 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36\nPUT /checkout HTTP/1.1 500 1943 Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Safari/605.1.15\nGET /products?id=2672 HTTP/1.1 302 1967 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1\nGET /products?id=8657 HTTP/1.1 200 1715 Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Safari/605.1.15\n\nQuestion: Which HTTP method was used most often for successful requests (status code 200)?\nAnswer: Request/Response method, except one, which is the Accept method. There are seve

## Vectorize: t5-small
## Generator: EleutherAI/gpt-neo-1.3B

In [19]:
t5_tokenizer = AutoTokenizer.from_pretrained("t5-small")
t5_model = T5EncoderModel.from_pretrained("t5-small")

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [20]:
processor_t5_gptNeo= LogProcessor(df,model_name="t5", tokenizer=t5_tokenizer, model=t5_model)

index_t5_gptNeo, chunked_logs_t5_gptNeo = processor_t5_gptNeo.process_no_logits()
query = queries[-1]
chunked_query_t5_gptNeo = processor_t5_gptNeo.chunk_text(query)
retrieved_logs_t5_gptNeo = processor_t5_gptNeo.retrieve_logs_no_logits(chunked_query_t5_gptNeo, index_t5_gptNeo, chunked_logs_t5_gptNeo)

In [21]:
processor_t5_gptNeo.generate_answer(retrieved_logs_t5_gptNeo, query, generator_gptNeo)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Context: GET /checkout HTTP/1.1 302 3329 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1\nDELETE /checkout HTTP/1.1 200 2014 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36\nDELETE /checkout HTTP/1.1 302 4739 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1\nDELETE /checkout HTTP/1.1 302 200 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36\nDELETE /checkout HTTP/1.1 404 1972 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36\n\nQuestion: Which HTTP method was used most often for successful requests (status code 200)?\nAnswer: 302\n\nA:\n\nI would try sending the following headers:\n-d status=301\n-d version=2.0\n...\n\nYou may h

## Vectorize: EleutherAI/gpt-neo-1.3B
## Generator: EleutherAI/gpt-neo-1.3B

In [22]:
gptNeo_tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")
gptNeo_model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B")

In [23]:
processor_gptNeo_gptNeo= LogProcessor(df,model_name="gptNeo", tokenizer=gptNeo_tokenizer, model=gptNeo_model)

index_gptNeo_gptNeo, chunked_logs_gptNeo_gptNeo = processor_gptNeo_gptNeo.process_logits()
query = queries[-1]
chunked_query_gptNeo_gptNeo = processor_gptNeo_gptNeo.chunk_text(query)
retrieved_logs_gptNeo_gptNeo = processor_gptNeo_gptNeo.retrieve_logs_logits(chunked_query_gptNeo_gptNeo, index_gptNeo_gptNeo, chunked_logs_gptNeo_gptNeo)

In [24]:
processor_gptNeo_gptNeo.generate_answer(retrieved_logs_gptNeo_gptNeo, query, generator_gptNeo)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Context: POST /contact HTTP/1.1 200 1995 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1\nPUT /checkout HTTP/1.1 302 1994 Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Safari/605.1.15\nPOST /contact HTTP/1.1 500 1996 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1\nPUT /contact HTTP/1.1 500 2005 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1\nPUT /index.html HTTP/1.1 500 2003 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1\n\nQuestion: Which HTTP method was used most often for successful requests (status code 200)?\nAnswer: HTTP GET.\nI assume the problem is on my end.\nThe question 

## Vectorize: roberta-base
## Generator: EleutherAI/gpt-neo-1.3B

In [25]:
roberta_tokenizer = AutoTokenizer.from_pretrained("roberta-base")
roberta_model = AutoModelForCausalLM.from_pretrained("roberta-base")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

If you want to use `RobertaLMHeadModel` as a standalone, add `is_decoder=True.`


In [26]:
processor_roberta_gptNeo= LogProcessor(df,model_name="roberta", tokenizer=roberta_tokenizer, model=roberta_model)

index_roberta_gptNeo, chunked_logs_roberta_gptNeo = processor_roberta_gptNeo.process_logits()
query = queries[-1]
chunked_query_roberta_gptNeo = processor_roberta_gptNeo.chunk_text(query)
retrieved_logs_roberta_gptNeo = processor_roberta_gptNeo.retrieve_logs_logits(chunked_query_roberta_gptNeo, index_roberta_gptNeo, chunked_logs_roberta_gptNeo)

In [27]:
processor_roberta_gptNeo.generate_answer(retrieved_logs_roberta_gptNeo, query, generator_gptNeo)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Context: PUT /contact HTTP/1.1 200 3075 Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0\nPUT /search HTTP/1.1 200 4983 Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0\nPOST /cart HTTP/1.1 200 2029 Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0\nPOST /cart HTTP/1.1 200 2060 Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0\nPOST /cart HTTP/1.1 200 1087 Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0\n\nQuestion: Which HTTP method was used most often for successful requests (status code 200)?\nAnswer: request.method == "GET"\n\nA:\n\nI think you should put your form data in a form variable:\ndef get_cart():\n        cart = request.form[\'cart\']\n '

# Testing with questions

In [28]:
def question_no_logits(query,processor, index, chunked_logs, generator):
    chunked_query = processor.chunk_text(query)
    retrieved_logs = processor.retrieve_logs_no_logits(chunked_query, index, chunked_logs)
    answer= processor.generate_answer(retrieved_logs, query, generator)
    print(answer)

def question_logits(query,processor, index, chunked_logs, generator): 
    chunked_query = processor.chunk_text(query)
    retrieved_logs = processor.retrieve_logs_logits(chunked_query, index, chunked_logs)
    answer = processor.generate_answer(retrieved_logs, query, generator)
    print(answer)

### How many requests resulted in a 404 status code?

In [29]:
question_no_logits(queries[0], processor_l6_gptNeo, index_l6_gptNeo,chunked_logs_l6_gptNeo, generator_gptNeo)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: POST /about HTTP/1.1 404 1670 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36
POST /about HTTP/1.1 404 4018 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36
POST /about HTTP/1.1 404 3001 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36
POST /about HTTP/1.1 404 4301 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36
GET /about HTTP/1.1 404 1886 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36

Question: How many requests resulted in a 404 status code?
Answer: There was only one 404.




In [30]:
question_no_logits(queries[0], processor_bert_gptNeo, index_bert_gptNeo,chunked_logs_bert_gptNeo, generator_gptNeo)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: DELETE /contact HTTP/1.1 404 3973 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
DELETE /about HTTP/1.1 404 4769 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
DELETE /about HTTP/1.1 404 1922 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
GET /products?id=8562 HTTP/1.1 404 2489 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
DELETE /about HTTP/1.1 500 4963 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1

Question: How many requests resulted in a 404 status code?
Answer: Only one. The problem is: there isn't a 404 status code. (The http 404 is

In [31]:
question_no_logits(queries[0], processor_distilbert_gptNeo, index_distilbert_gptNeo,chunked_logs_distilbert_gptNeo, generator_gptNeo)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: GET /products?id=6417 HTTP/1.1 302 1928 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
DELETE /checkout HTTP/1.1 404 1773 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
DELETE /checkout HTTP/1.1 404 2889 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
DELETE /checkout HTTP/1.1 404 4618 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
DELETE /checkout HTTP/1.1 404 4115 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1

Question: How many requests resulted in a 404 status code?
Answer: I think the number of 404 requests should be zero, but that's o

In [32]:
question_no_logits(queries[0], processor_t5_gptNeo, index_t5_gptNeo,chunked_logs_t5_gptNeo, generator_gptNeo)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: DELETE /checkout HTTP/1.1 404 1945 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36
DELETE /products HTTP/1.1 404 1874 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36
DELETE /products HTTP/1.1 404 731 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36
DELETE /products HTTP/1.1 404 4319 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36
GET /products?id=6137 HTTP/1.1 404 1909 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1

Question: How many requests resulted in a 404 status code?
Answer: I was wondering how many GET's the /product filter is filtering, in order to get the current count.

A:

You could have a look at this line of code for the coun

In [33]:
question_logits(queries[0], processor_gptNeo_gptNeo, index_gptNeo_gptNeo,chunked_logs_gptNeo_gptNeo, generator_gptNeo)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: POST /contact HTTP/1.1 200 1995 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
POST /contact HTTP/1.1 500 1996 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
PUT /contact HTTP/1.1 500 2005 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
PUT /index.html HTTP/1.1 500 2003 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
GET /products?id=9008 HTTP/1.1 500 1986 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1

Question: How many requests resulted in a 404 status code?
Answer: 200 was the only response that was returned which was not due to server er

In [34]:
question_logits(queries[0], processor_roberta_gptNeo, index_roberta_gptNeo,chunked_logs_roberta_gptNeo, generator_gptNeo)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: POST /cart HTTP/1.1 200 2029 Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0
POST /cart HTTP/1.1 404 898 Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0
POST /cart HTTP/1.1 200 2060 Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0
POST /cart HTTP/1.1 404 3940 Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0
PUT /cart HTTP/1.1 404 3048 Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0

Question: How many requests resulted in a 404 status code?
Answer: 1

A:

I don't think you're correctly calling the function "getCategories"
I'll suggest a few things. 
First, add the variable $var_categories to your code.
Next, the


### What is the most common HTTP method?

In [35]:
question_no_logits(queries[1], processor_l6_gptNeo, index_l6_gptNeo,chunked_logs_l6_gptNeo, generator_gptNeo)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: GET /about HTTP/1.1 500 1780 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36
GET /about HTTP/1.1 500 1998 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36
GET /about HTTP/1.1 302 4152 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36
GET /about HTTP/1.1 302 471 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36
GET /about HTTP/1.1 500 1834 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36

Question: What is the most common HTTP method?
Answer: All

A:

The most common HTTP method of course is GET. The server is telling the client that it needs some data from the request.
Your browser is parsing the request and is sending the response.
Other methods are used


In [36]:
question_no_logits(queries[1], processor_bert_gptNeo, index_bert_gptNeo,chunked_logs_bert_gptNeo, generator_gptNeo)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: GET /products?id=8562 HTTP/1.1 404 2489 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
GET /user/profile HTTP/1.1 404 4513 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
GET /about HTTP/1.1 404 2272 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
DELETE /login HTTP/1.1 302 2208 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
PUT /about HTTP/1.1 404 2928 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1

Question: What is the most common HTTP method?
Answer: GET, POST, PUT, DELETE, HEAD, OPTIONS, TRACE, TRACK, OPTIONS  (HEAD and TRACE are also su

In [37]:
question_no_logits(queries[1], processor_distilbert_gptNeo, index_distilbert_gptNeo,chunked_logs_distilbert_gptNeo, generator_gptNeo)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: GET /products?id=2747 HTTP/1.1 200 2298 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36
GET /products?id=6544 HTTP/1.1 200 345 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36
GET /products?id=5688 HTTP/1.1 500 3148 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36
GET /products?id=5327 HTTP/1.1 404 4763 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36
GET /products?id=1371 HTTP/1.1 200 1210 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36

Question: What is the most common HTTP method?
Answer: POST and GET.

A:

What is the most common HTTP method?

PUT, DELETE

As you may have suspected, the HTTP methods by default are POST and GET, but the HTTP methods can be chang

In [38]:
question_no_logits(queries[1], processor_t5_gptNeo, index_t5_gptNeo,chunked_logs_t5_gptNeo, generator_gptNeo)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: DELETE /checkout HTTP/1.1 200 2014 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36
DELETE /about HTTP/1.1 302 4939 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36
DELETE /checkout HTTP/1.1 302 4739 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
DELETE /checkout HTTP/1.1 302 4907 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36
DELETE /checkout HTTP/1.1 302 724 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36

Question: What is the most common HTTP method?
Answer: POST
I checked all the existing POST requests on this site: http://www.htc.org/htdocs/tools/request_tools.php. One thing that stands out is POST requests with a 302:
POST /checkout

In [39]:
question_logits(queries[1], processor_gptNeo_gptNeo, index_gptNeo_gptNeo,chunked_logs_gptNeo_gptNeo, generator_gptNeo)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: POST /contact HTTP/1.1 200 1995 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
POST /contact HTTP/1.1 500 1996 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
PUT /contact HTTP/1.1 500 2005 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
PUT /index.html HTTP/1.1 500 2003 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
GET /products?id=9008 HTTP/1.1 500 1986 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1

Question: What is the most common HTTP method?
Answer: GET.  GET is the most common method (there are several other possible answers).
Exampl

In [40]:
question_logits(queries[1], processor_roberta_gptNeo, index_roberta_gptNeo,chunked_logs_roberta_gptNeo, generator_gptNeo)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: PUT /contact HTTP/1.1 200 3075 Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0
PUT /search HTTP/1.1 200 4983 Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0
POST /cart HTTP/1.1 200 2060 Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0
DELETE /login HTTP/1.1 200 4082 Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0
DELETE /login HTTP/1.1 200 2343 Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0

Question: What is the most common HTTP method?
Answer: GET. The most common HTTP method is GET.

HTTP/1.1 200 3075 Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0)
  Geo/0.


### What is the most common status code?

In [41]:
question_no_logits(queries[2], processor_l6_gptNeo, index_l6_gptNeo,chunked_logs_l6_gptNeo, generator_gptNeo)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: POST /about HTTP/1.1 500 1623 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36
POST /about HTTP/1.1 500 2041 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36
POST /about HTTP/1.1 500 3309 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36
POST /about HTTP/1.1 500 744 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36
POST /about HTTP/1.1 500 982 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36

Question: What is the most common status code?
Answer: 0 for all requests, but for some requests the status code can be different. 

Note: It is not possible for us to answer your question by saying "what is the most common status code" because we cannot know for a fact the


In [42]:
question_no_logits(queries[2], processor_bert_gptNeo, index_bert_gptNeo,chunked_logs_bert_gptNeo, generator_gptNeo)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: DELETE /contact HTTP/1.1 500 4661 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
DELETE /about HTTP/1.1 500 4963 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
DELETE /contact HTTP/1.1 302 442 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
DELETE /contact HTTP/1.1 500 339 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
DELETE /about HTTP/1.1 302 4745 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1

Question: What is the most common status code?
Answer: 200 (OK) (or any other status code)

Question: What is the most specific status code?
Answe

In [43]:
question_no_logits(queries[2], processor_distilbert_gptNeo, index_distilbert_gptNeo,chunked_logs_distilbert_gptNeo, generator_gptNeo)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: GET /products?id=7127 HTTP/1.1 200 4747 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
GET /products?id=6417 HTTP/1.1 302 1928 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
GET /products?id=7173 HTTP/1.1 302 4923 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
GET /products?id=4726 HTTP/1.1 200 4395 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
GET /products?id=2672 HTTP/1.1 302 1967 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1

Question: What is the most common status code?
Answer: 302 on any HTTP status code, because the 302 response h

In [44]:
question_no_logits(queries[2], processor_t5_gptNeo, index_t5_gptNeo,chunked_logs_t5_gptNeo, generator_gptNeo)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: GET /products?id=7173 HTTP/1.1 302 4923 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
DELETE /checkout HTTP/1.1 302 4739 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
DELETE /checkout HTTP/1.1 200 2014 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36
DELETE /checkout HTTP/1.1 302 4953 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36
DELETE /checkout HTTP/1.1 302 4907 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36

Question: What is the most common status code?
Answer: 302
I've seen this many different ways. Some people see it as an indication that the server is done processing the request. Other people think it's

In [45]:
question_logits(queries[2], processor_gptNeo_gptNeo, index_gptNeo_gptNeo,chunked_logs_gptNeo_gptNeo, generator_gptNeo)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: POST /contact HTTP/1.1 200 1995 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
POST /contact HTTP/1.1 500 1996 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
PUT /contact HTTP/1.1 500 2005 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
PUT /index.html HTTP/1.1 500 2003 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
GET /products?id=9008 HTTP/1.1 500 1986 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1

Question: What is the most common status code?
Answer: It depends...
I'm thinking the answer will be different for different web servers (apa

In [46]:
question_logits(queries[2], processor_roberta_gptNeo, index_roberta_gptNeo,chunked_logs_roberta_gptNeo, generator_gptNeo)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: PUT /contact HTTP/1.1 200 3075 Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0
PUT /search HTTP/1.1 200 4983 Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0
DELETE /login HTTP/1.1 200 4082 Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0
PUT /login HTTP/1.1 500 3035 Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0
DELETE /login HTTP/1.1 200 2343 Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0

Question: What is the most common status code?
Answer:
200 - ok
301 - no content
404 - no such resource
500 - server error

A:

404 - no such resource

This looks like an error (as you're not viewing the resource on your browser)


### What is the most common user agent?

In [47]:
question_no_logits(queries[3], processor_l6_gptNeo, index_l6_gptNeo,chunked_logs_l6_gptNeo, generator_gptNeo)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: POST /user/profile HTTP/1.1 500 763 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
POST /user/profile HTTP/1.1 302 2657 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
POST /user/profile HTTP/1.1 200 2017 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
POST /user/profile HTTP/1.1 302 1890 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
POST /user/profile HTTP/1.1 500 599 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1

Question: What is the most common user agent?
Answer: Google/Chrome/Safari

A:

Google Chrome is used by most mobile users, Saf

In [48]:
question_no_logits(queries[3], processor_bert_gptNeo, index_bert_gptNeo,chunked_logs_bert_gptNeo, generator_gptNeo)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: DELETE /about HTTP/1.1 302 2882 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
DELETE /contact HTTP/1.1 500 339 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
DELETE /about HTTP/1.1 302 3269 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
DELETE /about HTTP/1.1 500 3356 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
DELETE /about HTTP/1.1 302 4745 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1

Question: What is the most common user agent?
Answer: Android.

A:

"What is the most common user agent" is a very broad question, so let me try to h

In [49]:
question_no_logits(queries[3], processor_distilbert_gptNeo, index_distilbert_gptNeo,chunked_logs_distilbert_gptNeo, generator_gptNeo)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: GET /products?id=6417 HTTP/1.1 302 1928 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
GET /products?id=8562 HTTP/1.1 404 2489 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
GET /products?id=6028 HTTP/1.1 200 4077 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
GET /products?id=3262 HTTP/1.1 404 2899 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
GET /products?id=3607 HTTP/1.1 200 1833 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1

Question: What is the most common user agent?
Answer: Apple's UA is Safari

A:

The most common user agent is 

In [50]:
question_no_logits(queries[3], processor_t5_gptNeo, index_t5_gptNeo,chunked_logs_t5_gptNeo, generator_gptNeo)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: DELETE /user/profile HTTP/1.1 500 2322 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36
DELETE /checkout HTTP/1.1 200 2014 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36
DELETE /user/profile HTTP/1.1 200 2218 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36
DELETE /user/profile HTTP/1.1 200 2421 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36
DELETE /user/profile HTTP/1.1 500 1871 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36

Question: What is the most common user agent?
Answer:
Google Chrome

A:

Chrome has a preference pane that shows the user agent in the "Advanced" tab. It can change the user agent to "Mozilla/5.0 (Windows NT 5.1; en


In [51]:
question_logits(queries[3], processor_gptNeo_gptNeo, index_gptNeo_gptNeo,chunked_logs_gptNeo_gptNeo, generator_gptNeo)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: POST /contact HTTP/1.1 200 1995 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
POST /contact HTTP/1.1 500 1996 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
PUT /contact HTTP/1.1 500 2005 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
PUT /index.html HTTP/1.1 500 2003 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
GET /products?id=9008 HTTP/1.1 500 1986 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1

Question: What is the most common user agent?
Answer:
The most common browser is Firefox.
Browser: IE (Internet explorer), Safari, Google Chr

In [52]:
question_logits(queries[3], processor_roberta_gptNeo, index_roberta_gptNeo,chunked_logs_roberta_gptNeo, generator_gptNeo)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: DELETE /login HTTP/1.1 200 2343 Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0
DELETE /login HTTP/1.1 200 4082 Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0
PUT /login HTTP/1.1 500 3035 Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0
PUT /login HTTP/1.1 200 2348 Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0
PUT /login HTTP/1.1 500 2938 Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0

Question: What is the most common user agent?
Answer: It is Chrome.
Question: How does it work?
Answer: Your browser sends the following HTTP headers:
Accept-Language: en, en-US, en-GB, en-GB-u-a-v, en-


### Which User Agent was used most frequently?

In [53]:
question_no_logits(queries[4], processor_l6_gptNeo, index_l6_gptNeo,chunked_logs_l6_gptNeo, generator_gptNeo)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: GET /user/profile HTTP/1.1 302 1299 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
GET /user/profile HTTP/1.1 302 1433 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
GET /user/profile HTTP/1.1 500 492 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
GET /user/profile HTTP/1.1 302 273 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
GET /user/profile HTTP/1.1 200 2011 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1

Question: Which User Agent was used most frequently?
Answer: 
Mozilla/5.0 (X11; Linux i686; rv:2.0.1.2pre3-s7) Gecko/20100101 Firefo

In [54]:
question_no_logits(queries[4], processor_bert_gptNeo, index_bert_gptNeo,chunked_logs_bert_gptNeo, generator_gptNeo)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: DELETE /about HTTP/1.1 302 2882 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
GET /products?id=8562 HTTP/1.1 404 2489 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
DELETE /about HTTP/1.1 302 3203 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
DELETE /checkout HTTP/1.1 302 2547 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
DELETE /about HTTP/1.1 302 4850 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1

Question: Which User Agent was used most frequently?
Answer: It was my personal preference to use the first one that would match the reques

In [55]:
question_no_logits(queries[4], processor_distilbert_gptNeo, index_distilbert_gptNeo,chunked_logs_distilbert_gptNeo, generator_gptNeo)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: GET /products?id=6417 HTTP/1.1 302 1928 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
GET /products?id=8562 HTTP/1.1 404 2489 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
GET /products?id=3262 HTTP/1.1 404 2899 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
GET /products?id=3607 HTTP/1.1 200 1833 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
GET /products?id=2672 HTTP/1.1 302 1967 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1

Question: Which User Agent was used most frequently?
Answer: It was "Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 l

In [56]:
question_no_logits(queries[4], processor_t5_gptNeo, index_t5_gptNeo,chunked_logs_t5_gptNeo, generator_gptNeo)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: GET /products?id=7127 HTTP/1.1 200 4747 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
DELETE /user/profile HTTP/1.1 500 2322 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36
DELETE /user/profile HTTP/1.1 200 2218 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36
DELETE /user/profile HTTP/1.1 404 1854 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36
DELETE /user/profile HTTP/1.1 500 1871 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36

Question: Which User Agent was used most frequently?
Answer:
  +---------------------------------------------------------------------+
  | User Agent: Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X)   |


In [57]:
question_logits(queries[4], processor_gptNeo_gptNeo, index_gptNeo_gptNeo,chunked_logs_gptNeo_gptNeo, generator_gptNeo)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: POST /contact HTTP/1.1 200 1995 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
POST /contact HTTP/1.1 500 1996 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
PUT /contact HTTP/1.1 500 2005 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
PUT /index.html HTTP/1.1 500 2003 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
GET /products?id=9008 HTTP/1.1 500 1986 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1

Question: Which User Agent was used most frequently?
Answer: User Agent was used most frequently, it was a Chrome. When we switch to Opera, S

In [58]:
question_logits(queries[4], processor_roberta_gptNeo, index_roberta_gptNeo,chunked_logs_roberta_gptNeo, generator_gptNeo)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: PUT /contact HTTP/1.1 200 2778 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
PUT /cart HTTP/1.1 500 2973 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
PUT /contact HTTP/1.1 500 3748 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
PUT /contact HTTP/1.1 200 2059 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
PUT /contact HTTP/1.1 404 1449 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1

Question: Which User Agent was used most frequently?
Answer: Chrome.

A:

It's not really a user agent problem, its a user's browser problem which causes issu

### What is the average response size for requests to /user/profile?

In [59]:
question_no_logits(queries[5], processor_l6_gptNeo, index_l6_gptNeo,chunked_logs_l6_gptNeo, generator_gptNeo)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: POST /user/profile HTTP/1.1 500 2605 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36
POST /user/profile HTTP/1.1 200 1679 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36
POST /user/profile HTTP/1.1 302 313 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36
POST /user/profile HTTP/1.1 200 2208 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36
POST /user/profile HTTP/1.1 302 703 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36

Question: What is the average response size for requests to /user/profile?
Answer: I'd suspect that the max response size would be at least that seen here.

However, /user/profile is a pretty large, complex page. Is there a simple way to find the 

In [60]:
question_no_logits(queries[5], processor_bert_gptNeo, index_bert_gptNeo,chunked_logs_bert_gptNeo, generator_gptNeo)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: GET /products?id=8562 HTTP/1.1 404 2489 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
DELETE /user/profile HTTP/1.1 500 4043 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
GET /products?id=4683 HTTP/1.1 404 1452 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
GET /user/profile HTTP/1.1 404 4513 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
PUT /user/profile HTTP/1.1 404 4512 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1

Question: What is the average response size for requests to /user/profile?
Answer:

The average response size for reque

In [61]:
question_no_logits(queries[5], processor_distilbert_gptNeo, index_distilbert_gptNeo,chunked_logs_distilbert_gptNeo, generator_gptNeo)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: GET /products?id=7127 HTTP/1.1 200 4747 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
GET /products?id=6417 HTTP/1.1 302 1928 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
GET /products?id=2460 HTTP/1.1 200 4436 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
GET /products?id=4745 HTTP/1.1 200 3100 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
GET /products?id=3607 HTTP/1.1 200 1833 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1

Question: What is the average response size for requests to /user/profile?
Answer: When I checked the Network 

In [62]:
question_no_logits(queries[5], processor_t5_gptNeo, index_t5_gptNeo,chunked_logs_t5_gptNeo, generator_gptNeo)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: DELETE /checkout HTTP/1.1 200 2014 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36
DELETE /user/profile HTTP/1.1 404 4312 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36
DELETE /user/profile HTTP/1.1 404 1854 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36
DELETE /user/profile HTTP/1.1 200 2421 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36
DELETE /user/profile HTTP/1.1 200 1412 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36

Question: What is the average response size for requests to /user/profile?
Answer: According to the spec, the average response size is 0.7 KB. 
Is there a way to calculate the average response size?

A:

The average response size is determ

In [63]:
question_logits(queries[5], processor_gptNeo_gptNeo, index_gptNeo_gptNeo,chunked_logs_gptNeo_gptNeo, generator_gptNeo)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: POST /contact HTTP/1.1 200 1995 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
POST /contact HTTP/1.1 500 1996 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
PUT /contact HTTP/1.1 500 2005 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
PUT /index.html HTTP/1.1 500 2003 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
GET /products?id=9008 HTTP/1.1 500 1986 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1

Question: What is the average response size for requests to /user/profile?
Answer: 1,000 KB

A:

In your request you are sending 10% of the t

In [64]:
question_logits(queries[5], processor_roberta_gptNeo, index_roberta_gptNeo,chunked_logs_roberta_gptNeo, generator_gptNeo)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: PUT /user/profile HTTP/1.1 200 3175 Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0
PUT /user/profile HTTP/1.1 200 4561 Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0
PUT /user/profile HTTP/1.1 200 2529 Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0
PUT /user/profile HTTP/1.1 302 4961 Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0
PUT /user/profile HTTP/1.1 200 4150 Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0

Question: What is the average response size for requests to /user/profile?
Answer: The average response size for requests to /user/profile is 5 bytes. As of the time of this question the average response size is not tracked anywhere.

A:

The client does not send a size, they are simply content-


### How many unique URLs were accessed?

In [65]:
question_no_logits(queries[6], processor_l6_gptNeo, index_l6_gptNeo,chunked_logs_l6_gptNeo, generator_gptNeo)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: GET /about HTTP/1.1 404 571 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36
GET /about HTTP/1.1 500 1834 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36
GET /about HTTP/1.1 404 4227 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36
GET /about HTTP/1.1 404 1592 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36
GET /about HTTP/1.1 404 1788 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36

Question: How many unique URLs were accessed?
Answer: 
1.6 billion

A:

The short answer is that Chrome is sending back at each GET (the entire page) all the data it received on request.
The complete answer is 

 The answer is in the second


In [66]:
question_no_logits(queries[6], processor_bert_gptNeo, index_bert_gptNeo,chunked_logs_bert_gptNeo, generator_gptNeo)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: DELETE /about HTTP/1.1 404 4769 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
GET /products?id=8562 HTTP/1.1 404 2489 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
GET /products?id=4683 HTTP/1.1 404 1452 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
DELETE /search HTTP/1.1 404 2064 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
DELETE /checkout HTTP/1.1 404 4115 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1

Question: How many unique URLs were accessed?
Answer: The first query was found 0 times within one minute. The second query was fo

In [67]:
question_no_logits(queries[6], processor_distilbert_gptNeo, index_distilbert_gptNeo,chunked_logs_distilbert_gptNeo, generator_gptNeo)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: GET /products?id=6417 HTTP/1.1 302 1928 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
GET /products?id=8562 HTTP/1.1 404 2489 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
DELETE /checkout HTTP/1.1 404 4618 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
DELETE /checkout HTTP/1.1 404 4115 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
DELETE /checkout HTTP/1.1 404 1326 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1

Question: How many unique URLs were accessed?
Answer: 6

Question: What kind of information was requested?
Answer: A list of 

In [68]:
question_no_logits(queries[6], processor_t5_gptNeo, index_t5_gptNeo,chunked_logs_t5_gptNeo, generator_gptNeo)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: GET /products?id=7173 HTTP/1.1 302 4923 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
DELETE /checkout HTTP/1.1 200 2014 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36
DELETE /products HTTP/1.1 200 244 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36
DELETE /cart HTTP/1.1 200 1912 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36
GET /products?id=6137 HTTP/1.1 404 1909 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1

Question: How many unique URLs were accessed?
Answer: 4,086 unique requests

Question: How many unique URLs were accessed?
Answer: 4,086 unique requests

Question: How many unique URLs were accessed?
An

In [69]:
question_logits(queries[6], processor_gptNeo_gptNeo, index_gptNeo_gptNeo,chunked_logs_gptNeo_gptNeo, generator_gptNeo)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: POST /contact HTTP/1.1 200 1995 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
POST /contact HTTP/1.1 500 1996 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
PUT /contact HTTP/1.1 500 2005 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
PUT /index.html HTTP/1.1 500 2003 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
GET /products?id=9008 HTTP/1.1 500 1986 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1

Question: How many unique URLs were accessed?
Answer: It looks like the server returned all the links in the same order. However, there is a 

In [70]:
question_logits(queries[6], processor_roberta_gptNeo, index_roberta_gptNeo,chunked_logs_roberta_gptNeo, generator_gptNeo)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: PUT /contact HTTP/1.1 200 3075 Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0
PUT /search HTTP/1.1 200 4983 Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0
PUT /login HTTP/1.1 404 3088 Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0
PUT /cart HTTP/1.1 200 1539 Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0
PUT /cart HTTP/1.1 404 3048 Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0

Question: How many unique URLs were accessed?
Answer: 0

A:

You could try using curl_multi instead of cURL.
As for the "hundreds of thousands of unique URLs accessed":
You're using curl_multi to create multiple calls, which is a poor solution


### Which HTTP method was used most often for successful requests (status code 200)?

In [71]:
question_no_logits(queries[7], processor_l6_gptNeo, index_l6_gptNeo,chunked_logs_l6_gptNeo, generator_gptNeo)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: GET /about HTTP/1.1 302 1016 Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0
GET /about HTTP/1.1 302 1135 Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0
GET /about HTTP/1.1 500 1998 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36
GET /about HTTP/1.1 200 4019 Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0
GET /about HTTP/1.1 500 1834 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36

Question: Which HTTP method was used most often for successful requests (status code 200)?
Answer: GET.

A:

According to RFC 2616,

The “Location” header and the “Upgrade” header together define
  the preferred method for fetching the resource, with the “Transfer-


In [72]:
question_no_logits(queries[7], processor_bert_gptNeo, index_bert_gptNeo,chunked_logs_bert_gptNeo, generator_gptNeo)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: DELETE /about HTTP/1.1 302 2882 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
DELETE /about HTTP/1.1 302 3203 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
DELETE /checkout HTTP/1.1 302 2547 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
DELETE /settings HTTP/1.1 404 2674 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
DELETE /about HTTP/1.1 302 754 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1

Question: Which HTTP method was used most often for successful requests (status code 200)?
Answer: POST
I've been browsing through the PHP logs a

In [73]:
question_no_logits(queries[7], processor_distilbert_gptNeo, index_distilbert_gptNeo,chunked_logs_distilbert_gptNeo, generator_gptNeo)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: GET /products?id=6417 HTTP/1.1 302 1928 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
PUT /checkout HTTP/1.1 404 1985 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36
PUT /checkout HTTP/1.1 500 1943 Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Safari/605.1.15
GET /products?id=2672 HTTP/1.1 302 1967 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
GET /products?id=8657 HTTP/1.1 200 1715 Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Safari/605.1.15

Question: Which HTTP method was used most often for successful requests (status code 200)?
Answer: POST

Question: Which HTTP methods did you use most frequently?
Answer: GET

Question: 

In [74]:
question_no_logits(queries[7], processor_t5_gptNeo, index_t5_gptNeo,chunked_logs_t5_gptNeo, generator_gptNeo)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: GET /checkout HTTP/1.1 302 3329 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
DELETE /checkout HTTP/1.1 200 2014 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36
DELETE /checkout HTTP/1.1 302 4739 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
DELETE /checkout HTTP/1.1 302 200 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36
DELETE /checkout HTTP/1.1 404 1972 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36

Question: Which HTTP method was used most often for successful requests (status code 200)?
Answer: The GET method.

A:

The answer is simple, because the request URL and the request bodies are the same 
http://e

In [75]:
question_logits(queries[7], processor_gptNeo_gptNeo, index_gptNeo_gptNeo,chunked_logs_gptNeo_gptNeo, generator_gptNeo)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: POST /contact HTTP/1.1 200 1995 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
PUT /checkout HTTP/1.1 302 1994 Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Safari/605.1.15
POST /contact HTTP/1.1 500 1996 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
PUT /contact HTTP/1.1 500 2005 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1
PUT /index.html HTTP/1.1 500 2003 Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1

Question: Which HTTP method was used most often for successful requests (status code 200)?
Answer: POST, 302, 500... but most of time was 302.

A:

I've actually done 

In [76]:
question_logits(queries[7], processor_roberta_gptNeo, index_roberta_gptNeo,chunked_logs_roberta_gptNeo, generator_gptNeo)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: PUT /contact HTTP/1.1 200 3075 Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0
PUT /search HTTP/1.1 200 4983 Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0
POST /cart HTTP/1.1 200 2029 Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0
POST /cart HTTP/1.1 200 2060 Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0
POST /cart HTTP/1.1 200 1087 Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0

Question: Which HTTP method was used most often for successful requests (status code 200)?
Answer: GET /payment HTTP/1.1 200 4983 Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0
PUT /
